In [1]:
import pandas as pd
import numpy as np
import os
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, desc
from sqlalchemy.ext.automap import automap_base
import json
from pprint import pprint
from sqlalchemy import inspect

from flask import Flask, jsonify, render_template
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, desc 
import pandas as pd
import json



In [2]:
engine = create_engine("sqlite:///Merged_Drivers_Data.sqlite")
session = Session(bind = engine)

In [5]:
Base = declarative_base()
# reflect the tables
Base.metadata.create_all(engine)

In [18]:
inspector = inspect(engine)

In [22]:

inspector.get_table_names()
columns = inspector.get_columns('Accidents')

In [23]:
for column in columns:
    print(column["name"], column["type"])

index INTEGER
STATE INTEGER
ST_CASE INTEGER
VE_TOTAL INTEGER
VE_FORMS INTEGER
PVH_INVL INTEGER
PEDS INTEGER
PERNOTMVIT INTEGER
PERMVIT INTEGER
PERSONS INTEGER
COUNTY INTEGER
CITY INTEGER
DAY INTEGER
MONTH INTEGER
YEAR INTEGER
DAY_WEEK INTEGER
HOUR INTEGER
MINUTE INTEGER
NHS INTEGER
RUR_URB INTEGER
FUNC_SYS INTEGER
RD_OWNER INTEGER
ROUTE INTEGER
TWAY_ID TEXT
TWAY_ID2 TEXT
MILEPT INTEGER
LATITUDE REAL
LONGITUD REAL
SP_JUR INTEGER
HARM_EV INTEGER
MAN_COLL INTEGER
RELJCT1 INTEGER
RELJCT2 INTEGER
TYP_INT INTEGER
WRK_ZONE INTEGER
REL_ROAD INTEGER
LGT_COND INTEGER
WEATHER1 INTEGER
WEATHER2 INTEGER
WEATHER INTEGER
SCH_BUS INTEGER
RAIL TEXT
NOT_HOUR INTEGER
NOT_MIN INTEGER
ARR_HOUR INTEGER
ARR_MIN INTEGER
HOSP_HR INTEGER
HOSP_MN INTEGER
CF1 INTEGER
CF2 INTEGER
CF3 INTEGER
FATALS INTEGER
DRUNK_DR INTEGER


In [45]:
deaths_per_hundred_million_miles_traveled = session.query(merged_drivers_data.state,merged_drivers_data.deaths_per_hundred_million_miles_traveled).all()
deaths_df = pd.DataFrame(deaths_per_hundred_million_miles_traveled)


with open('states.json') as f:
    data = json.load(f)

    state_coordinates = []
    states = []

for x in range(len(data['features'])):
    state_coordinates.append(data['features'][x]['geometry'])
    states.append(data['features'][x]['properties']['name'])

    geojson_df = pd.DataFrame({'state':states,'geometry':state_coordinates,'type':'Feature'})
    result_df = geojson_df.merge(deaths_df,on="state")
    result_dict = {"features":(result_df.to_dict()),"type":"FeatureCollection"}
    key_change = result_dict['features']['deaths_per_hundred_million_miles_traveled']
    for key,value in key_change.items():
        new_key = str(key_change)
        key_change[new_key] = key_change.pop(key)
    return key_change

In [90]:
b = {"a": 'b'}
a = {"a": b}
id(a["a"]) == id(b)

True

In [29]:
deaths_per_hundred_million_miles_traveled = session.query(merged_drivers_data.state,merged_drivers_data.deaths_per_hunderd_thousand,merged_drivers_data.population).all()
deaths_df = pd.DataFrame(deaths_per_hundred_million_miles_traveled)
deaths_df.drop(deaths_df[deaths_df['state'] == "District of Columbia"].index, inplace=True)
def region(c):
    if c['state'] in ['Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 'Rhode Island', 'Vermont','New Jersey','New York','Pennsylvania']:
        return 'Northeast'
    elif c['state'] in ['Illinois','Indiana','Michigan','Ohio','Wisconsin','Iowa','Kansas','Minnesota','Missouri','Nebraska','North Dakota','South Dakota']:
        return 'Midwest'
    elif c['state'] in ['Delaware','Florida','Georgia','Maryland','North Carolina','South Carolina','Virginia','West Virginia','Alabama','Kentucky','Mississippi','Tennessee','Arkansas','Louisiana','Oklahoma','Texas']:
        return 'South'
    else:
        return 'West'
    
deaths_df['Region'] = deaths_df.apply(region,axis=1)

deaths_df

,state,deaths_per_hunderd_thousand,population,Region
0,Alabama,21.3,4863300,South
1,Alaska,11.3,741894,West
2,Arizona,13.9,6931071,West
3,Arkansas,18.2,2988248,South
4,California,9.2,39250017,West
5,Colorado,11.0,5540545,West
6,Connecticut,8.2,3576452,Northeast
7,Delaware,12.5,952065,South
9,Florida,15.4,20612439,South
10,Georgia,15.1,10310371,South


In [36]:
deaths_df.drop(deaths_df[deaths_df['state'] = "District of Columbia"].index, inplace=True)


SyntaxError: invalid syntax (<ipython-input-36-44a8389d951f>, line 1)

In [34]:
def region(c):
    if c['state'] in ['Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 'Rhode Island', 'Vermont','New Jersey','New York','Pennsylvania']:
        return 'Northeast'
    elif c['state'] in ['Illinois','Indiana','Michigan','Ohio','Wisconsin','Iowa','Kansas','Minnesota','Missouri','Nebraska','North Dakota','South Dakota']:
        return 'Midwest'
    elif c['state'] in ['Delaware','Florida','Georgia','Maryland','North Carolina','South Carolina','Virginia','West Virginia','Alabama','Kentucky','Mississippi','Tennessee','Arkansas','Louisiana','Oklahoma','Texas']:
        return 'South'
    else:
        return 'West'
    
deaths_df['Region'] = deaths_df.apply(region,axis=1)

deaths_df.groupby('Region').sum()

,deaths_per_hunderd_thousand,population
Region,,
Midwest,141.8,67941429
Northeast,72.1,56209510
South,254.2,121638404
West,165.3,76657000


In [13]:
deaths_miles = session.query(merged_drivers_data.state,merged_drivers_data.deaths_per_hundred_million_miles_traveled).all()
d
deaths_df = pd.DataFrame(deaths_miles, columns=('state','deaths_per_hundred_million_miles_traveled'))

In [14]:
deaths_df

,state,deaths_per_hundred_million_miles_traveled
0,Alabama,1.49
1,Alaska,1.67
2,Arizona,1.44
3,Arkansas,1.51
4,California,0.99
5,Colorado,1.15
6,Connecticut,0.92
7,Delaware,1.17
8,District of Columbia,0.76
9,Florida,1.44


In [39]:
deaths_df.describe()

,deaths_per_hunderd_thousand,population
count,50.000000,5.000000e+01
mean,12.668000,6.448927e+06
std,4.385851,7.271769e+06
min,4.800000,5.855010e+05
25%,9.225000,1.850106e+06
50%,12.250000,4.559320e+06
75%,15.325000,7.198768e+06
max,23.100000,3.925002e+07
